In [1]:
print('Analysis Dashboard')

Analysis Dashboard


In [2]:
from ipywidgets import interact, interactive, widgets, interact_manual
import glob
import pandas as pd
import plotly.graph_objects as go


from scipy.signal import savgol_filter
import peakutils

from ipyfilechooser import FileChooser

In [4]:
class fileselection():
    def __init__(self):
        fc = FileChooser('/srv/scratch/')
        display(fc)

        fc.use_dir_icons = True
        fc.title = '<b>Choose Folder</b>'
        fc.register_callback(self.change_title)
    def change_title(self, chooser):
        chooser.title = '<b>Callback function executed</b>'
        print('ahai')

    # Register callback function
vid = fileselection()

FileChooser(path='/srv/scratch', filename='', show_hidden='False')

ahai


In [ ]:
%%html
<a href="https://costa.jack.engineering/user/jack/tree/scratch" target=_blank>Files</a>

In [3]:


class fileselection():
    def __init__(self):
        self.folders = glob.glob('Data/tracking_generated/*')
        self.filepath = None
        
        for i in range(len(self.folders)):
            self.folders[i] = self.folders[i].split('/')[2]

        display(interactive(self.loadfiles, Folders=self.folders))

    def choosevideo(self, Data):
        global dataframe, origin
        self.filepath = Data
        self.dataframe = self.origin = pd.read_csv(Data)
        #print(self.origin)

    def loadfiles(self, Folders):
        files = sorted(glob.glob('Data/tracking_generated/' + Folders + '/*.csv'))
        display(interactive(self.choosevideo, Data=files))
vid = fileselection()

interactive(children=(Dropdown(description='Folders', options=('Untitled Folder',), value='Untitled Folder'), …

In [4]:

def plot(thresh, peakdist, buffer, polynomial, window):
    peaks = []
    basepoints = []
    data = abs(vid.origin)
    data['disp'] = savgol_filter(vid.origin['disp'], window, polynomial)
    data['disp'] = data['disp'] * -1
    peaks.append(peakutils.indexes(data['disp'], thresh, peakdist))
    peaks[0] = peaks[0][1:-1]
    basepoints.append([])
    for peak in peaks[0]:
        for k in range(peak - buffer, 1, -1):
            dfdt = (data['disp'][k] - data['disp'][k - 1])
            if dfdt<=0:
                basepoints[0].append(k)
                break

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['time'],y=data['disp'])) 
    fig.add_trace(go.Scatter(x=data['time'][peaks[0]], y=data['disp'][peaks[0]], mode = 'markers'))
    fig.add_trace(go.Scatter(x=data['time'][basepoints[0]], y=data['disp'][basepoints[0]], mode = 'markers'))
    fig.show()


In [5]:
@interact(thres = (0, 1, .1), peakdist = (0, 30), buffer = (0, 20), poly = (1, 16), window = (9, 31, 2))
def paramaterization(thres = .6, peakdist = 10, buffer = 5, poly = 3, window = 13):
    plot(thres, peakdist, buffer, poly, window)
    #return (x, y, z, k, c)

interactive(children=(FloatSlider(value=0.6, description='thres', max=1.0), IntSlider(value=10, description='p…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=3, description='Polynomial', max=16, min=1), IntSlider(value=13, descrip…

interactive(children=(FloatSlider(value=0.6, description='Thresh', max=1.0), IntSlider(value=10, description='…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

TypeError: __init__() got an unexpected keyword argument 'x'